#### I'm always curious about *Cryptography*!
- Also, this one is also based on *Python* 3.7.0.
- Here's the [link](https://docs.python.org/3/library/hashlib.html).

#### Hash Algorithms

In [1]:
import hashlib 

m = hashlib.sha256()

m.update(b'hello')
m.update(b' motherfucker')

m
m.name         # can be passed as a parameter to 'new()' (don't use it..)

m.digest()
m.hexdigest()

m.digest_size  # the size of resulting hash (bytes)
m.block_size   # the internal block size of the hash algorithm

<sha256 HASH object @ 0x105e7f1c0>

'sha256'

b'\xb5\xf1\x8am\xafb\x06J\x9c\x072\xa0U\xc1\x10\x8c~D\x13\xc9\xba\xc0\xb8\x1b\xc8\xfb\x9a\xe2\xcdZo\xb3'

'b5f18a6daf62064a9c0732a055c1108c7e4413c9bac0b81bc8fb9ae2cd5a6fb3'

32

64

In [2]:
algo_available = hashlib.algorithms_available
algo_guarantee = hashlib.algorithms_guaranteed

len(algo_available)  # available in the running Python interpreter 
len(algo_guarantee)  # means 'supported on ALL platforms'

32

14

In [3]:
q = hashlib.sha224()

# bytes
q.update(b'hello')

# u(a);u(b) => u(a+b)
q.update(b'hashlib is fun')

q.digest()     # contains bytes from 0 to 255 
q.hexdigest()  # string object 

b'\xa0J\xe8$\xa0\xa5U\xe4\xeb\xdcb\x1e|\xa9XX\x18\xe3\x8e.\xde\xf4?\xa7\x83[\x0f"'

'a04ae824a0a555e4ebdc621e7ca9585818e38e2edef43fa7835b0f22'

#### Key derivation

In [4]:
import hashlib, binascii

dk = hashlib.pbkdf2_hmac(
    'sha256',     # algorithms 
    b'123456',    # password
    b'xxx',       # "salt" (should be >16 bytes & with random obj)          
    1000,         # iterations 
    20            # the length of the derived key
)

# hexadecimal representation of bin data
binascii.hexlify(dk)

b'c30d41309b87c19900fea94bb41c60601d6000f0'

<hr>
BLAKE2 -- another algorithm (same level as MD5, SHA-1 etc.)
<hr>

#### example - simple hashing

In [5]:
from hashlib import blake2b

In [6]:
# basic procedures 

bp = blake2b()             # create hash object
bp.update(b'hello blake')  # feed data

bp.digest()                # show it 
bp.hexdigest()             # another way 

b'5~\xfcq\x1f\xd94\xef\x80<*@\x18\xe2\x8f\xaaS\xady\x892\r\n\x11\x8c\x87\x99@\xac\x1f\xc9=\xf4\\u\x16\xf1\xf6\xc9\x80\x12@\x84\x8e,#:\xf5N\xfd(=\x8e\xd5\x03^\x03|\xdc\xe8b\xf0\x92\xb6'

'357efc711fd934ef803c2a4018e28faa53ad7989320d0a118c879940ac1fc93df45c7516f1f6c9801240848e2c233af54efd283d8ed5035e037cdce862f092b6'

In [7]:
# do it in one line! 
blake2b(b'one line').hexdigest()

'ea518cfece1201a856a114e0c52123700d6ae061101c1e1eb1a275d97bda600064dbc8aaadebf6070b6cf2ed3e86513767910479a2c33918b7fbf196e58f994b'

In [8]:
# u can 'update' many times 

um = blake2b()

for item in [b'you',b'are',b'awesome']:
    um.update(item)
    
um.hexdigest()

'898e6354eae8489b0dd786f7c1503b1953bde59e928c04246d9a0296eb24cd2e5c96820f0be901d0e73c1358f4d7aef9df19bfc74a0ec1c267d089c9492bd31b'

#### example - digest size

In [9]:
# specifying digest sizes 

ds = blake2b(digest_size=10)
ds.update(b'hey guys')

ds.hexdigest()

ds.digest()
ds.digest_size, len(ds.digest())

'75d38b15cccd8657a5b5'

b'u\xd3\x8b\x15\xcc\xcd\x86W\xa5\xb5'

(10, 10)

In [10]:
# objs with different digest-size 
#   have completely different outputs.

blake2b(digest_size=10).hexdigest()
blake2b(digest_size=12).hexdigest()

'6fa1d8fcfd719046d762'

'b8e1dda3ac0aa3820ad2990b'

#### example - keyed hashing

In [11]:
kh = blake2b(key=b'pseudorandom key!',digest_size=16)

kh.update(b'get a hex-coded 128b auth code')

kh.digest()
kh.hexdigest()

b'\x86\xb4\xe4\x02\x11\x13b5\x16\xa7F\xd0\xa1\xa3\xe3\xc1'

'86b4e4021113623516a746d0a1a3e3c1'

In [12]:
''' A practical example (cookie stuff) '''

from hashlib import blake2b
from hmac import compare_digest 

SECURE_KEY = b'random generated secrect key'
AUTH_SIZE  = 16

def sign(cookie):
    h = blake2b(digest_size=AUTH_SIZE, key=SECURE_KEY)
    h.update(cookie)
    return h.hexdigest().encode('utf-8')

def verify(cookie, sig):
    good_sig = sign(cookie)
    return compare_digest(good_sig, sig)


cookie = b'user-John'
sig    = sign(cookie)  # encode it 

# John - before/after encoding 
"{0}, {1}".format(cookie.decode('utf-8'),sig)

verify(cookie, sig)          # plain-text(J) / encoded(J)
verify(b'user-Bob', sig)     # other-text(B) / encoded(J)

verify(cookie, b'32193182')  # plain-text(J) / encoded(Other)

' A practical example (cookie stuff) '

"user-John, b'0a68d1517ea621fbd338eb76f11a8f09'"

True

False

False

In [13]:
import hmac
import hashlib 

# construct with 'hmac' module
m = hmac.new(b'secrect key', digestmod=hashlib.blake2s)

m.update(b'a message')
m.hexdigest()

'038d7996ebe18c0a0390595e1575c4dcbe25de93d5fdf238fa81ede665314304'

#### example - randomized hashing

In [14]:
import os 
from hashlib import blake2b

tips = ''' 
    Randomized hashing is used for 
    protecting against collision attacks on the hash function used in digital signatures.
    
    There's a warning (standard library):
        Salted hashing (or just hashing) with BLAKE2 or any other 
        general-purpose cryptographic hash func, such as SHA-256, 
        is NOT suitable for hashing passwords.
'''

salt_one = os.urandom(blake2b.SALT_SIZE)
salt_two = os.urandom(blake2b.SALT_SIZE)

rh1 = blake2b(salt=salt_one)
rh1.update(b'some message')

rh2 = blake2b(salt=salt_two)
rh2.update(b'some message')

# NOT the same (cuz the random)
rh1.hexdigest()
rh2.hexdigest()

'8f944b4fd34a77d5cde18ea00af092df114b9e217d75704ce31e8c21265e8fba97c340f7fe3df01c2c96d35884e08f622bac4ebd298c91f81c0eac118dc278a6'

'9ee0b3573926d41ef28d0673f4eb7cae62ec99c344a83bbd4f3f2efd4455fee559ff286b68f26a71476983df5a57f05dfe4d2ed072583cae8c5c948602cc9492'

#### example - personalization

In [15]:
from hashlib import blake2b

''' Adding attribute: "person" '''

FILES_HASH_PERSON = b'myapp Files hash'
BLOCK_HASH_PERSON = b'myapp Block hash'

hp1 = blake2b(digest_size=32, person=FILES_HASH_PERSON)
hp1.update(b'same content')

hp2 = blake2b(digest_size=32, person=BLOCK_HASH_PERSON)
hp2.update(b'same content')

hp1.hexdigest()
hp2.hexdigest()

' Adding attribute: "person" '

'a7da3398cca486f7e0d03ec22a837ee67b97328a42fd15363f4c404888a3f6a2'

'd29c972cf442ab3aad75500f1348b767c06496a1e7a0d95caf4440343f96977d'

In [16]:
from hashlib import blake2s
from base64 import b64encode, b64decode

tips = '''
    Personalization with the keyed mode can also 
    be used to [derived different keys from a single one].
'''

# copied from standard library 
_key = b'Rm5EPJai72qcK3RGBpW3vPNfZy5OZothY+kHY6h21KM='

orig_key = b64decode(_key)

# hashing & showing it 
enc_key = blake2s(key=orig_key, person=b'kEncrypt').digest()
mac_key = blake2s(key=orig_key, person=b'kMAC').digest()

# encode the b'xxx' with base64 
b64encode(enc_key).decode('utf-8') 
b64encode(mac_key).decode('utf-8')

'rbPb15S/Z9t+agffno5wuhB77VbRi6F9Iv2qIxU7WHw='

'G9GtHFE1YluXY1zWPlYk1e/nWfu0WSEb0KRcjhDeP/o='

#### example - tree mode

In [17]:
#  10
#  / \
# 00 01

from hashlib import blake2b

# data

FANOUT     = 2
DEPTH      = 2 
LEAF_SIZE  = 4096
INNER_SIZE = 64

buf = bytearray(6000)

In [18]:
# Left leaf 
h00 = blake2b(
    buf[0:LEAF_SIZE], 
    
    fanout=FANOUT,
    depth=DEPTH,
    leaf_size=LEAF_SIZE, 
    inner_size = INNER_SIZE,
    
    node_offset=0, 
    node_depth=0, 
    last_node=False 
)

# Right leaf 
h01 = blake2b(
    buf[LEAF_SIZE:], 
    
    fanout=FANOUT,
    depth=DEPTH,
    leaf_size=LEAF_SIZE, 
    inner_size = INNER_SIZE,
    
    node_offset=1, 
    node_depth=0, 
    last_node=True 
)

# Root leaf 
h10 = blake2b(
    digest_size=32,
    
    fanout=FANOUT,
    depth=DEPTH,
    leaf_size=LEAF_SIZE, 
    inner_size = INNER_SIZE,
    
    node_offset=0, 
    node_depth=1, 
    last_node=True 
)


h10.hexdigest()

h10.update(h00.digest())
h10.update(h01.digest())

h10.hexdigest()

'9c3d2354abe4248873c284440540b01aa3a11c52b659226e8c1e9cbd3fcc4a93'

'3ad2a9b37c6070e374c7a8c508fe20ca86b6ed54e286e93a0318e95e881db5aa'